In [29]:
import asyncio
import logging
import os
import warnings
from operator import itemgetter

import tiktoken
from langchain_teddynote.callbacks import StreamingCallback
from langchain import hub
from dotenv import load_dotenv
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import VLLM
import json
from langchain_community.chat_models import ChatOllama
from langchain_teddynote.messages import stream_response

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


In [30]:
# vllm 0.6.0
from langchain_community.llms import VLLM
llm = VLLM(
    model="beomi/Qwen2.5-7B-Instruct-kowiki-qa-context",
    # model="NCSOFT/Llama-VARCO-8B-Instruct",
    # model="Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4",
    trust_remote_code=True,  # Hugging Face 모델의 경우 필수
    max_new_tokens=512,
    top_k=5,
    top_p=0.8,
    temperature=0.7,
    dtype = "bfloat16",
    gpu_memory_utiliztion=0.5,
    repetition_penalty=1.1,
    skip_special_tokens=True,
    stop = ["}","}}"]
 )

INFO 11-22 17:21:02 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='beomi/Qwen2.5-7B-Instruct-kowiki-qa-context', speculative_config=None, tokenizer='beomi/Qwen2.5-7B-Instruct-kowiki-qa-context', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=beomi/Qwen2.5-7B-Instruct-kowiki-qa-context, use_v2_block_manager=False, num_scheduler

OutOfMemoryError: CUDA out of memory. Tried to allocate 260.00 MiB. GPU 0 has a total capacity of 23.64 GiB of which 135.06 MiB is free. Including non-PyTorch memory, this process has 23.46 GiB memory in use. Of the allocated memory 22.66 GiB is allocated by PyTorch, with 62.53 MiB allocated in private pools (e.g., CUDA Graphs), and 163.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7, streaming=True)

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler

class CustomStreamingCallbackHandler(BaseCallbackHandler):
    def __init__(self):
        self.partial_result = ""

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.partial_result += token
        print(token, end="", flush=True)  # 실시간 출력
        # '}'가 생성되면 중단
        if "}" in self.partial_result:
            raise StopIteration("중단 조건 '}'이 생성되어 종료합니다.")

In [ ]:
hf_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

In [ ]:
PARTIAL_SUMMARY_PROMPT_TEMPLATE = """Please summarize the sentence according to the following REQUEST.

                                        REQUEST:
                                        1. Summarize the main points in bullet points in KOREAN.
                                        2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
                                        3. Use various emojis to make the summary more interesting.
                                        4. Translate the summary into KOREAN if it is written in ENGLISH.
                                        5. DO NOT translate any technical terms.
                                        6. DO NOT include any unnecessary information.
                                        
                                        CONTEXT:
                                        {context}
                                        
                                        SUMMARY:"
                                        """
# FINAL_SUMMARY_PROMPT_TEMPLATE = """
# Please summarize the CONTEXT according to the following REQUEST and provide the output EXACTLY as shown in the example format below. 

# REQUEST:
# 1. DO NOT include any text or explanation outside the JSON format. The response must be valid JSON that can be parsed directly without errors.
# 2. Summarize the main points in bullet points in KOREAN, but DO NOT translate any technical terms.
# 3. Each summarized sentence must start with a single emoji that fits the meaning of the sentence.
# 4. Use various emojis to make the summary more interesting, but keep it concise and relevant.
# 5. Focus on identifying and presenting only one main topic and one overall summary for the document.
# 6. Avoid redundant or repeated points, and ensure that the summary covers all key ideas without introducing multiple conclusions or topics.
# 7. Please refer to each summary and indicate the key topic.
# 8. If the original text is in English, we have already provided a summary translated into Korean, so please do not provide a separate translation.
# 9. Based on the summarized content, please create the three most relevant recommended questions in KOREAN.

# CONTEXT:
# {context}

# IMPORTANT - CHECK AGAIN BEFORE RESPONDING:
# DO NOT include any text or explanation outside the JSON format. 
# The response must be valid JSON that can be parsed directly without errors.

# OUTPUT(JSON FORMAT):
# {{
#     "FINAL_SUMMARY": {{
#         "Key topic": [Key topic content],
#         "Summaries": [
#             • [Emoji] [Summary point],
#             ...
#         ]
#     }},
#     "RECOMMEND_QUESTIONS": [
#         [First question in KOREAN],
#         [Second question in KOREAN],
#         [Third question in KOREAN]
#     ]
# }}
# """
FINAL_SUMMARY_PROMPT_TEMPLATE = """
다음 REQUEST에 따라 CONTEXT를 요약하고, 출력은 아래에 제공된 출력 형식(OUTPUT_FORMAT)과 정확히 동일하게 한 번만 작성해주세요.

REQUEST:
1. 주어진 OUTPUT(JSON 형식) 외의 텍스트나 설명을 포함하지 마세요.
2. CONTEXT와 HUMAN MESSAGE는 출력하지마세요.
3. 단 하나의 OUTPUT만 출력하세요.
4. 주요 내용을 한국어로 요약하되, 전문, 기술 용어는 원본을 사용하세요.
5. 요약된 각 문장은 해당 의미와 잘 어울리는 이모지 하나로 시작해야 합니다.
6. 다양한 이모지를 사용하여 요약을 흥미롭게 작성하되, 간결하고 관련성 있게 유지하세요.
7. 문서의 단일 주요 주제와 전반적인 요약에만 집중하세요.
8. 각 요약에서 주요 주제를 명확히 나타내세요.
9. CONTEXT의 내용이 충분히 많다면, 요약 문장을 충분히 생성하세요.
10. 요약된 내용을 기반으로 가장 관련성 높은 세 가지 질문을 한국어로 작성하세요.

CONTEXT:
{context}

OUTPUT_FORMAT(JSON 형식):
{{
    "FINAL_SUMMARY": {{
        "Key_topic": 주요 주제 내용,
        "Summaries": [
            "• Emoji 요약된 내용1",
            "• Emoji 요약된 내용2",
            ...추가 요약 내용 나열
        ]
    }},
    "RECOMMEND_QUESTIONS": [
        "첫 번째 질문 (한국어)",
        "두 번째 질문 (한국어)",
        "세 번째 질문 (한국어)"
    ]
}}
OUTPUT:
"""



In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=10
)
summarize_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=500
)
partial_summary_prompt = PromptTemplate.from_template(
    PARTIAL_SUMMARY_PROMPT_TEMPLATE
)
final_summary_prompt = PromptTemplate.from_template(
    FINAL_SUMMARY_PROMPT_TEMPLATE
)

In [ ]:
# with open("../runpod_backend/data/backup.json","r") as f:
#     data = json.load(f)
# script = data["m25Lz9KWyDkNx6Vv"]["script_info"]["script"]
with open("./test1234/transcript.json","r") as f:
    script = json.load(f)

In [ ]:
documents = [
            Document(page_content="\n".join([t["text"] for t in script]))
        ]

In [31]:
partial_summary_chain = create_stuff_documents_chain(
            llm=llm, prompt=partial_summary_prompt
        )

In [32]:
map_summary = hub.pull("teddynote/map-summary-prompt")

In [33]:
map_summary.pretty_print()

================================ System Message ================================

You are an expert summarizer. Your task is to summarize the following document in {language}.

================================ Human Message =================================

Extract most important main thesis from the documents, then summarize in bullet points.

#Format:
- summary 1
- summary 2
- summary 3
-...

Here is a given document: 
{documents}

Write 1~5 sentences. Think step by step.
#Summary:


In [34]:
# map chain 생성
map_chain = map_summary | llm | StrOutputParser()

In [35]:
from langchain_core.runnables import chain



def map_refine_chain(docs):

    # map chain 생성
    map_summary = hub.pull("teddynote/map-summary-prompt")

    map_chain = (
        map_summary
        | llm
        | StrOutputParser()
    )

    input_doc = [{"documents": doc.page_content, "language": "Korean"} for doc in docs]

    # 첫 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    doc_summaries = map_chain.batch(input_doc)

    refine_prompt = hub.pull("teddynote/refine-prompt")


    refine_chain = refine_prompt | llm | StrOutputParser()

    previous_summary = doc_summaries[0]

    for current_summary in doc_summaries[1:]:

        previous_summary = refine_chain.invoke(
            {
                "previous_summary": previous_summary,
                "current_summary": current_summary,
                "language": "Korean",
            }
        )
        print("\n\n-----------------\n\n")

    return previous_summary


In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
summarize_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000, chunk_overlap=400
        )

In [37]:
def calculate_tokens(text, model="gpt-4o-mini"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)
calculate_tokens(documents[0].page_content)

5728

In [38]:
docs = summarize_splitter.split_documents(documents)

In [39]:
summary = map_refine_chain(docs)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.57s/it, est. speed input: 309.35 toks/s, output: 143.47 toks/s]




-----------------




Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.56s/it, est. speed input: 309.62 toks/s, output: 143.72 toks/s]




-----------------




Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.57s/it, est. speed input: 309.18 toks/s, output: 143.39 toks/s]


...



-----------------




Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.57s/it, est. speed input: 309.15 toks/s, output: 143.63 toks/s]




-----------------




Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.56s/it, est. speed input: 309.17 toks/s, output: 143.64 toks/s]



-----------------




In [40]:
print(summary)

 If the context is useful, integrate it into the summary.

Assistant: 딥러닝에서 규모가 모델 성능에 미치는 중요성과 병렬화 가능한 아키텍처의 이점을 설명하며, 관련 자료는 Andre Karpathy, Chris Olah, Vivek, 그리고 Britt Cruz의 동영상으로 제공된다. 어텐션 메커니즘은 쿼리와 키 벡터 간의 내적을 사용하여 단어들 간의 관련성을 측정하며, 차원의 제곱근으로 나눈 값은 숫자의 안정성을 높습니다. 소프트맥스 함수를 사용하여 주의 가중치를 계산하고, 이를 통해 어텐션 패턴을 생성합니다. 어텐션 패턴은 각 단어가 다른 단어에 대한 관련성을 가질 수 있도록 합니다. 모델은 훈련 과정에서 모든 가능한 다음 토큰을 예측하여 훈련 효율성을 높입니다. 또한, 이전 단어가 이후 단어에 영향을 미치지 않도록 주의 메커니즘이 일관되게 작동하도록 합니다. 이 문서는 주로 어텐션 메커니즘의 핵심 개념과 다양한 컨텍스트가 단어의 의미에 미치는 영향에 초점을 맞추고 있습니다. GPT-3와 같은 모델은 96개의 레이어를 통해 각 단어가 더 많은 문맥을 흡수하게 되어, 더 높은 수준의 추상적 개념을 인코딩할 수 있는 능력을 갖추게 됩니다. GPT-3의 총 매개변수는 약 580억개로, 이는 전체 매개변수의 약 30%에 불과합니다. 주요 매개변수의 대부분은 주어진 단계 사이의 블록들에서 옵니다. 병렬화가 가능하기 때문에, GPT-3는 빠른 시간 내에 많은 계산을 수행할 수 있습니다. 이 과정에서, GPT-3는 96개의 레이어를 통해 각 단어가 더 많은 문맥을 흡수하게 되어, 더 높


In [41]:
from pydantic import BaseModel, Field
from typing import List

class Summary(BaseModel):
    emoji: str = Field(..., description="요약에 사용하는 이모지")
    content: str = Field(..., description="요약된 내용")

class FinalSummary(BaseModel):
    key_topic: str = Field(..., description="주요 주제 내용")
    summaries: List[Summary] = Field(..., description="요약된 내용 리스트")

class RecommendQuestions(BaseModel):
    questions: List[str] = Field(..., description="추천 질문 리스트")

class FullStructure(BaseModel):
    FINAL_SUMMARY: FinalSummary = Field(..., description="최종 요약 정보")
    RECOMMEND_QUESTIONS: RecommendQuestions = Field(..., description="추천 질문 리스트")

In [42]:
parser = JsonOutputParser(pydantic_object=FullStructure)

In [43]:
final_summary_chain = create_stuff_documents_chain(
            llm=llm, prompt=final_summary_prompt, output_parser=parser
        )

In [44]:
partial_summary = [Document(page_content=summary)]

In [45]:
split_docs = summarize_splitter.split_documents(documents)

In [46]:
callback_handler = CustomStreamingCallbackHandler()

In [47]:
final_summary = final_summary_chain.invoke({"context": partial_summary})
print(final_summary)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.82s/it, est. speed input: 317.20 toks/s, output: 143.15 toks/s]

{'FINAL_SUMMARY': {'Key_topic': '어텐션 메커니즘과 병렬화 가능한 아키텍처', 'Summaries': ['🔍 어텐션 패턴 생성을 통해 단어들 간의 관련성을 측정', '📊 쿼리와 키 벡터의 내적을 통해 안정적인 숫자의 값 계산', '🔄 주의 메커니즘이 일관되게 작동하여 단어 간 영향 최소화', '🧠 GPT-3의 96개 레이어를 통해 더 많은 문맥 흡수', '⚡ 병렬화 가능한 아키텍처로 빠른 계산 수행', '📊 GPT-3의 580억개 매개변수 중 주요 매개변수의 대부분은 블록들에서']}, 'RECOMMEND_QUESTIONS': ['어텐션 메커니즘이 단어 간 관련성을 어떻게 측정하는지 자세히 설명해 주시겠습니까?', 'GPT-3의 병렬화 가능한 아키텍처가 모델의 성능에 어떤 영향을 미치는지 알려주실 수 있나요?', 'GPT-3의 580억개의 매개변수 중 주요 매개변수가 어떤 역할을 하는지 이해를 돕는 예를 들어 주실 수 있나요?']}


In [48]:
final_summary

{'FINAL_SUMMARY': {'Key_topic': '어텐션 메커니즘과 병렬화 가능한 아키텍처',
  'Summaries': ['🔍 어텐션 패턴 생성을 통해 단어들 간의 관련성을 측정',
   '📊 쿼리와 키 벡터의 내적을 통해 안정적인 숫자의 값 계산',
   '🔄 주의 메커니즘이 일관되게 작동하여 단어 간 영향 최소화',
   '🧠 GPT-3의 96개 레이어를 통해 더 많은 문맥 흡수',
   '⚡ 병렬화 가능한 아키텍처로 빠른 계산 수행',
   '📊 GPT-3의 580억개 매개변수 중 주요 매개변수의 대부분은 블록들에서']},
 'RECOMMEND_QUESTIONS': ['어텐션 메커니즘이 단어 간 관련성을 어떻게 측정하는지 자세히 설명해 주시겠습니까?',
  'GPT-3의 병렬화 가능한 아키텍처가 모델의 성능에 어떤 영향을 미치는지 알려주실 수 있나요?',
  'GPT-3의 580억개의 매개변수 중 주요 매개변수가 어떤 역할을 하는지 이해를 돕는 예를 들어 주실 수 있나요?']}

In [33]:
summary = ""
summary += f'{list(final_summary.get("FINAL_SUMMARY").items())[0][0]}:{list(final_summary.get("FINAL_SUMMARY").items())[0][1]}\n'
joined_summary = '\n'.join(list(final_summary.get("FINAL_SUMMARY").items())[1][1])
summary += f'{list(final_summary.get("FINAL_SUMMARY").items())[1][0]}:{joined_summary}'

In [34]:
summary

'Key_topic:Transformer의 주요한 이해: 어텐션 메커니즘\nSummaries:• 🔍 모델은 텍스트에서 다음 단어를 예측하기 위해 토큰을 고차원 벡터로 변환하며, 이 벡터는 단어의 의미를 인코딩한다.\n• 🧵 어텐션 메커니즘은 주변 토큰의 정보를 토큰 벡터에 통합하여 더 풍부한 맥락적 의미를 인코딩한다.\n• 🔍 각 토큰의 벡터는 고차원 공간에서 특정 방향으로 이동하여 단어의 다양한 의미를 인코딩한다.\n• 🔄 어텐션 헤드는 여러 가지 방향으로 벡터를 업데이트하여 단어의 의미를 보다 구체적으로 인코딩한다.\n• 📏 여러 어텐션 헤드가 병렬로 실행되어 모델은 다양한 맥락적 업데이트 방식을 학습할 수 있다.\n• 🔍 GPT-3는 96개의 어텐션 헤드를 사용하여 총 58억 개의 파라미터를 가진다.'

In [35]:
questions = final_summary.get("RECOMMEND_QUESTIONS")

In [36]:
print(summary)
print("------")
print(questions)

Key_topic:Transformer의 주요한 이해: 어텐션 메커니즘
Summaries:• 🔍 모델은 텍스트에서 다음 단어를 예측하기 위해 토큰을 고차원 벡터로 변환하며, 이 벡터는 단어의 의미를 인코딩한다.
• 🧵 어텐션 메커니즘은 주변 토큰의 정보를 토큰 벡터에 통합하여 더 풍부한 맥락적 의미를 인코딩한다.
• 🔍 각 토큰의 벡터는 고차원 공간에서 특정 방향으로 이동하여 단어의 다양한 의미를 인코딩한다.
• 🔄 어텐션 헤드는 여러 가지 방향으로 벡터를 업데이트하여 단어의 의미를 보다 구체적으로 인코딩한다.
• 📏 여러 어텐션 헤드가 병렬로 실행되어 모델은 다양한 맥락적 업데이트 방식을 학습할 수 있다.
• 🔍 GPT-3는 96개의 어텐션 헤드를 사용하여 총 58억 개의 파라미터를 가진다.
------
['Transformer의 어텐션 메커니즘이 어떻게 작동하는지 자세히 설명해 줄 수 있나요?', '어텐션 헤드가 여러 개 사용되는 이유는 무엇인가요?', 'GPT-3의 어텐션 메커니즘을 이해하기 위해선 어떤 주요한 개념을 알아야 하나요?']


In [37]:
documents[0].page_content += f"\n{summary}" 

In [38]:
split_docs = text_splitter.split_documents(documents)

In [39]:
vec_store = FAISS.from_documents(split_docs, hf_embeddings)
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k = 10
vec_retriever = vec_store.as_retriever(search_kwargs={"k": 10})
retriever = EnsembleRetriever(
                retrievers=[bm25_retriever, vec_retriever],
                weights=[0.7, 0.3],
            )

In [40]:
# # 사용자의 질문에 대한 답변
# response_schemas = [
#     ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
# ]
# # 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
# output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# format_instructions = output_parser.get_format_instructions()

In [41]:
prompt = PromptTemplate.from_template(
            """당신은 유튜브 스크립트 기반의 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다.
                당신의 주요 임무는 다음과 같습니다:
                1. 기본적으로 검색된 문맥(context)과 이전 대화 내용(chat_history)을 바탕으로 질문에 대해 OUTPUT_FORMAT의 형식을 반드시 지켜서 답변하세요.
                2. 주어진 OUTPUT(JSON 형식) 외의 텍스트나 설명을 포함하지 마세요.
                3. 다음과 같은 경우에는 자연스럽게 내부 지식을 활용하여 답변하세요:
                    - 검색된 문맥이 질문의 의도를 완벽하게 충족하지 못할 때
                    - 영상의 전반적인 주제와 연관되지만 구체적인 답변이 문맥에 없을 때
                    - 문맥에서 부분적인 정보만 찾을 수 있을 때는 문맥의 정보와 내부 지식을 조합하여 답변하세요

                4. 답변 시 다음 사항을 지켜주세요:
                - 항상 자연스러운 대화체로 답변하세요
                - 문맥에서 답을 찾을 수 없더라도, 그 사실을 언급하지 말고 바로 답변하세요
                - 기술 용어나 고유명사는 원어를 유지하세요
                - 전문적인 내용도 이해하기 쉽게 설명하세요

                5. 만약 질문이 영상의 주제나 내용과 전혀 관련이 없다면 "영상과 관계 없는 질문입니다."라고 답변하세요.

                이전 대화 내용:
                {chat_history}

                질문:
                {question}

                문맥:
                {context}

                OUTPUT_FORMAT:
                {{"answer": "답변"}}
                """
)

In [42]:
class chat_parser(BaseModel):
    answer: dict = Field(..., description="채팅 응답")

In [43]:
chain = (
                {
                    "context": itemgetter("question") | retriever,
                    "question": itemgetter("question"),
                    "chat_history": itemgetter("chat_history"),
                }
                | prompt
                # | chat_llm
                | llm
                | JsonOutputParser(pydantic_object=chat_parser)
                # | StrOutputParser()
            )

In [44]:
_message_store = {}
def _get_session_history(session_id: str) -> BaseChatMessageHistory:
        """세션 ID를 기반으로 대화 기록을 가져오는 함수"""
        if session_id not in _message_store:
            _message_store[session_id] = ChatMessageHistory()
        return _message_store[session_id]

In [45]:
chain_with_history = RunnableWithMessageHistory(
                chain,
                _get_session_history,
                input_messages_key="question",
                history_messages_key="chat_history",
            )

In [46]:
def chat(prompt):
    return chain_with_history.invoke(
                    {"question": prompt},
                    config={"configurable": {"session_id": "test1234"}},
                    callbacks = [callback_handler])

1. 트랜스포머의 어텐션 메커니즘은 어떻게 작동하며, 그 중요성은 무엇인가요?

2. 어텐션 메커니즘은 토큰의 고차원 벡터를 어떻게 조정하여 문맥에 따른 의미를 더 풍부하게 만드는가요?

3. 어텐션 메커니즘의 여러 헤드는 어떻게 작동하며, 그 이유는 무엇인가요? 

In [47]:
questions[0]

'Transformer의 어텐션 메커니즘이 어떻게 작동하는지 자세히 설명해 줄 수 있나요?'

In [49]:
print(chat('Transformer의 어텐션 메커니즘이 어떻게 작동하는지 간단하게 설명해 줄 수 있나요?'))

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.58s/it, est. speed input: 346.77 toks/s, output: 143.18 toks/s]

{'answer': 'Transformer의 어텐션 메커니즘은 주변 토큰의 정보를 토큰 벡터에 통합하여 더 풍부한 맥락적 의미를 인코딩합니다. 여러 어텐션 헤드가 병렬로 실행되어 모델은 다양한 맥락적 업데이트 방식을 학습할 수 있습니다. 각 토큰은 고유한 벡터와 연결되며, 이 벡터는 토큰이 주변 토큰의 정보를 인코딩하는 데 도움이 됩니다. 어텐션 메커니즘은 토큰 간의 관련성을 강조하고, 이를 통해 모델은 더 복잡한 의미를 인식할 수 있습니다.'}


In [ ]:
import asyncio
import logging
import os
import warnings
from operator import itemgetter

import tiktoken
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema.output_parser import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
summarize_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000, chunk_overlap=500
        )

In [ ]:
import json
with open("script.json", "r") as f:
    script_data = json.load(f)

In [ ]:
import os
os.makedirs("test1234",exist_ok=True)
with open("test1234/transcript.json", "w") as f:
    json.dump(script_data, f)

In [ ]:
documents = [
            Document(page_content="\n".join([t["text"] for t in script_data]))
        ]

In [ ]:
split_docs = summarize_splitter.split_documents(documents)

In [ ]:
vec_store = FAISS.from_documents(split_docs, embeddings)

In [ ]:
vec_store.save_local("test1234")

In [ ]:
vec_store_load = FAISS.load_local("test11", embeddings=embeddings, allow_dangerous_deserialization=True)


In [ ]:
split_docs

In [ ]:
bm25_retriever = BM25Retriever.from_documents(split_docs)

In [ ]:
text = "[FINAL SUMMARY]\nKey topic: 레그의 이해 및 정보 처리\n\n• 📚 레그의 비법노트에 도달하기 위한 과정이 필요하다.  \n• 🔄 반복 학습을 통해 레그의 기본 개념과 구현 방식을 이해해야 한다.  \n• ✏️ 레그는 최신 정보를 제공하고, 정보 참조를 통해 질문에 답변할 수 있는 AI이다.  \n• ⚙️ 사전학습된 정보와 최신 정보의 차이를 이해해야 하며, 정보의 흐름을 잃지 않도록 주의해야 한다.  \n• 🔍 효과적인 정보 처리를 위해 관련성 있는 정보의 페이지만 제공하는 것이 중요하다.  \n• 📄 문서의 특정 단락을 선택하고, 유사도를 계산하여 필요한 정보를 추출한다.  \n• 💡 인베딩 과정을 통해 문장을 수학적 표현으로 변환하고, 이를 바탕으로 정보 검색을 향상시킨다.\n\n[RECOMMEND QUESTIONS]\n1. 레그를 활용하여 최신 정보를 어떻게 효율적으로 제공할 수 있을까?\n2. 사전학습된 정보와 최신 정보의 활용 시 고려해야 할 요소는 무엇인가?\n3. 인베딩 과정이 정보 검색에 미치는 영향은 어떤 것들이 있을까?"

In [ ]:
text.split("[FINAL SUMMARY]")[1].split("[RECOMMEND QUESTIONS]")[0].strip("\n\n")

In [ ]:
text.split("[FINAL SUMMARY]")[1].split("[RECOMMEND QUESTIONS]")[0].strip("\n\n").replace("\n\n", "\n").replace("\n\n", "\n")

In [ ]:
import streamlit as st

st.container?

In [ ]:
from pytubefix import YouTube
import re


class YouTubeService:
    async def get_title_and_hashtags(self, url: str):
        yt = await _create_youtube_instance(url)
        print("영상 정보 확인")
        title = yt.title
        description = yt.description
        hashtags = re.findall(r"#\w+", description)
        return {"title": title, "hashtags": " ".join(hashtags)}

    async def get_video_info(self, url: str):
        yt = await _create_youtube_instance(url)
        audio_stream = yt.streams.filter(only_audio=True).first()
        print("음성 추출 완료")
        return {
            "title": yt.title,
            "audio_url": audio_stream.url if audio_stream else None,
        }

    async def _create_youtube_instance(self, url: str):
        print("YouTube 인스턴스 생성 완료")
        return YouTube(url)


In [ ]:
import concurrent.futures
import math
import os
import tempfile
from typing import Any, Dict, List

import ffmpeg
import requests
import soundfile as sf
from faster_whisper import BatchedInferencePipeline, WhisperModel
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from konlpy.tag import Okt

class WhisperTranscriptionService:
    def __init__(self):
        model = WhisperModel(
            "large-v3", device="cuda", compute_type="float16"
        )
        model = BatchedInferencePipeline(model=model)
        language = None
        okt = Okt()
        print("Whisper 모델 초기화 완료")

    def create_session(self):
        session = requests.Session()
        retry = Retry(
            total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504]
        )
        adapter = HTTPAdapter(max_retries=retry, pool_connections=100, pool_maxsize=100)
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        return session

    def download_chunk(self, args):
        url, start, end, chunk_number, temp_dir = args

        headers = {"Range": f"bytes={start}-{end}"}
        session = create_session()

        try:
            response = session.get(url, headers=headers, stream=True)
            chunk_path = os.path.join(temp_dir, f"chunk_{chunk_number:04d}")

            with open(chunk_path, "wb") as f:
                for data in response.iter_content(chunk_size=8192):
                    f.write(data)

            return chunk_path, chunk_number
        except Exception as e:
            print(f"Error downloading chunk {chunk_number}: {str(e)}")
            return None, chunk_number

    def _single_stream_download(self, url: str, temp_dir: str) -> str:
        """단일 스트림으로 파일을 다운로드합니다."""
        print("Starting single stream download...")
        session = create_session()
        output_path = os.path.join(temp_dir, "complete_audio.mp4")

        try:
            with session.get(url, stream=True) as response:
                response.raise_for_status()
                with open(output_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
            return output_path
        except Exception as e:
            raise Exception(f"Failed to download file: {str(e)}")

    def parallel_download(self, url: str, temp_dir: str, num_chunks: int = 10) -> str:
        """병렬 다운로드를 시도하고, 실패 시 단일 스트림으로 폴백"""
        session = create_session()

        try:
            # HEAD 요청으로 파일 크기 확인 시도
            response = session.head(url, allow_redirects=True)
            total_size = int(response.headers.get("content-length", 0))

            # HEAD 요청이 실패하면 GET 요청으로 시도
            if total_size == 0:
                response = session.get(url, stream=True)
                total_size = int(response.headers.get("content-length", 0))

            # 파일 크기를 여전히 확인할 수 없는 경우 단일 스트림으로 다운로드
            if total_size == 0:
                print(
                    "Warning: Could not determine file size. Falling back to single stream download."
                )
                return _single_stream_download(url, temp_dir)
            print("Starting parallel download...")
            chunk_size = total_size // num_chunks
            chunks = []

            for i in range(num_chunks):
                start = i * chunk_size
                end = start + chunk_size - 1 if i < num_chunks - 1 else total_size - 1
                chunks.append((start, end))

            download_args = [
                (url, start, end, i, temp_dir) for i, (start, end) in enumerate(chunks)
            ]

            chunk_paths = []
            with concurrent.futures.ThreadPoolExecutor(
                max_workers=num_chunks
            ) as executor:
                futures = executor.map(download_chunk, download_args)
                chunk_paths = [(path, num) for path, num in futures if path is not None]

            if not chunk_paths:
                raise Exception("No chunks were downloaded successfully")

            chunk_paths.sort(key=lambda x: x[1])
            output_path = os.path.join(temp_dir, "complete_audio.mp4")

            with open(output_path, "wb") as outfile:
                for chunk_path, _ in chunk_paths:
                    with open(chunk_path, "rb") as infile:
                        outfile.write(infile.read())
                    os.remove(chunk_path)

            return output_path

        except Exception as e:
            print(
                f"Error in parallel download: {str(e)}. Falling back to single stream download."
            )
            return _single_stream_download(url, temp_dir)

    def convert_to_wav(self, input_path: str, output_path: str) -> bool:
        try:
            stream = ffmpeg.input(input_path)
            stream = ffmpeg.output(
                stream, output_path, acodec="pcm_s16le", ar="16000", ac="1"
            )
            ffmpeg.run(stream, capture_stdout=True, capture_stderr=True)
            return True
        except ffmpeg.Error as e:
            print("FFmpeg error:", e.stderr.decode())
            return False

    def process_audio_chunk(self, chunk_data: tuple,promp:str = None,filtered_words:list = None) -> List[Dict[str, Any]]:
        audio_path, start_time, duration = chunk_data
        try:
            segments, info = model.transcribe(
                audio_path,
                beam_size=5,
                best_of=7,
                batch_size=32,
                temperature=0.7,
                word_timestamps=True,
                initial_prompt=f"음성 제목: {promp}",
                repetition_penalty=2,
                no_repeat_ngram_size=3,
                length_penalty=1.1,
                log_prob_threshold=-0.5,
                no_speech_threshold=0.7,
                patience=1.2,
                hotwords=filtered_words
            )
            if info and hasattr(info, "language"):
                language = info.language
            return _process_segments(segments, start_time)
        except Exception as e:
            print(f"Error processing chunk at {start_time}: {str(e)}")
            return []

    def _process_segments(
        self, segments, start_time: float = 0
    ) -> List[Dict[str, Any]]:
        transcript = []
        for segment in segments:
            transcript.append(
                {
                    "start": round(segment.start + start_time, 2),
                    "end": round(segment.end + start_time, 2),
                    "text": segment.text,
                }
            )
        return transcript

    async def process_with_progress(
        self, url: str, prompt:str, filtered_words:str,chunk_duration: int = 30, num_download_chunks: int = 10
    ) -> List[Dict[str, Any]]:
        with tempfile.TemporaryDirectory() as temp_dir:
            mp4_path = parallel_download(url, temp_dir, num_download_chunks)
            print("Download complete!")

            wav_path = os.path.join(temp_dir, "audio.wav")
            if not convert_to_wav(mp4_path, wav_path):
                raise Exception("Failed to convert audio to WAV format")

            wav_info = sf.info(wav_path)
            total_duration = wav_info.duration
            total_chunks = math.ceil(total_duration / chunk_duration)

            chunks_data = []
            for i in range(total_chunks):
                start_time = i * chunk_duration
                chunk_wav_path = os.path.join(temp_dir, f"chunk_{i}.wav")

                duration = min(chunk_duration, total_duration - start_time)
                stream = ffmpeg.input(wav_path, ss=start_time, t=duration)
                stream = ffmpeg.output(
                    stream, chunk_wav_path, acodec="pcm_s16le", ar="16000", ac="1"
                )
                ffmpeg.run(stream, quiet=True)

                chunks_data.append((chunk_wav_path, start_time, duration))

            all_segments = []
            for chunk_data in chunks_data:
                segments = process_audio_chunk(chunk_data,prompt,filtered_words)
                all_segments.extend(segments)

                if os.path.exists(chunk_data[0]):
                    os.remove(chunk_data[0])

        return all_segments

    async def transcribe(self, audio_url: str,prompt: str = None) -> Dict[str, Any]:
        try:
            try:
                tagged = okt.pos(prompt)
                filtered_words = []
                for word, tag in tagged:
                    if tag == "Noun" or tag == "Hashtag":
                        filtered_words.append(word)
            except:
                filtered_words = None
            segments = await process_with_progress(
                audio_url, prompt, filtered_words,chunk_duration=30, num_download_chunks=10
            )

            print("텍스트 추출 완료")

            return {"script": segments, "language": language}
        except Exception as e:
            print(f"Error in transcribe: {str(e)}")
            raise


In [ ]:
youtube = YouTubeService()

In [ ]:
video_info = await youtube.get_video_info("https://youtu.be/EMMC0ym0QOI?si=bx7raBo-QwR3MGy7")

In [ ]:
video_info["audio_url"]

In [ ]:
video_info["title"]

In [ ]:
whisper = WhisperTranscriptionService()

In [ ]:
transcript = await whisper.transcribe(video_info["audio_url"],video_info["title"])

In [ ]:
for script in transcript["script"]:
    print(script["text"],"\n")

In [ ]:
transcript = await whisper.transcribe(video_info["audio_url"])

In [ ]:
for script in transcript["script"]:
    print(script["text"],"\n")

In [ ]:
print("### 주제: 레그의 개념 및 구현 방법\n\n- 📚 레그 비법노트를 통해 레그의 기본 개념과 구현 방식에 대한 이해가 중요합니다.\n- 🔍 실습 파일들을 반복적으로 검토하여 이해도를 높여야 합니다.\n- ❓ 레그의 주요 목적은 최신 정보를 포함한 정확한 답변을 제공하는 것입니다.\n- 🆚 GPT는 사전학습된 정보에 의존하지만, 레그는 제공된 정보를 바탕으로 더 정확한 답변을 생성합니다.\n- 📊 오래된 정보는 정확한 답변을 방해하며, 관련성 있는 정보만 제공하는 것이 최선입니다.\n- 📑 문맥을 통해 질문에 대한 답변을 찾고, 유사도 계산을 통해 관련 단락을 추출합니다.\n- 🔄 텍스트는 특정 키워드로 분할되어야 하며, 청크 오버랩을 통해 정보의 일관성을 유지합니다.\n- 💾 인베딩 과정을 통해 각 단락을 숫자 표현으로 변환하고 저장하여 나중에 검색할 수 있습니다.\n- 📊 인베딩 이해 후, 데이터를 저장해야 하며, 이 과정에서 비용이 발생합니다.\n- 📚 다음 영상에서는 레그의 후반부 내용을 다룰 예정입니다.\n\n### 추천 질문:\n1. 레그의 구현 방식에서 가장 중요한 요소는 무엇인가요?\n2. 인베딩 과정에서 발생하는 비용은 어떻게 관리할 수 있나요?\n3. 관련성 있는 정보를 효과적으로 추출하기 위한 방법은 무엇인가요?")

In [ ]:
text = "### KEY TOPIC: 레그의 기능과 정보 접근 방식\n\n- 📚 레그의 비법노트에 도착하기까지 수고하셨습니다.  \n- 🔄 레그에 대한 이해를 위해 반복 학습이 필요합니다.  \n- 🎨 레그의 목적을 시각적으로 설명할 예정입니다.  \n- ❓ 레그는 최신 정보를 제공하기 위해 사용됩니다.  \n- 📰 기존 채찍 PT와 비교하여 정보 접근 방식을 설명합니다.  \n- ⚙️ 프롬프트의 변화로 레그의 기능이 향상됩니다.  \n- 📄 PDF와 같은 자료를 활용하여 질문에 답변함.  \n- 🔑 관련성 있는 정보만 제공하는 것이 최선임.  \n- 📏 특정 단락만 필요한 경우 청크 사이즈를 설정하여 분할함.  \n- 🔍 유사도 계산을 통해 관련 단락을 뽑아냅니다.  \n- 📊 임베딩은 문자열을 수학적 표현으로 변환하는 과정임.  \n- 🔗 동일한 숫자 개수로 유사도 계산 가능함.  \n- 💰 인베딩 과정에서 비용이 발생하며, 많은 문서를 처리할 때 신중해야 함.  \n- 📽️ 이번 영상은 전처리 단계를 다루었고, 다음 영상에서는 후반부를 다룰 예정.\n\n### RECOMMENDED QUESTIONS:\n1. 레그의 정보 접근 방식에서 가장 중요한 요소는 무엇인가요?\n2. 임베딩 과정에서 발생하는 비용을 줄일 수 있는 방법은 무엇인가요?\n3. 레그의 기능을 향상시키기 위한 반복 학습의 필요성은 어떤 점에서 중요한가요?"

In [ ]:
print(text)

In [ ]:
print("### Key Topic: RAG (Retrieval-Augmented Generation) 시스템의 구현 및 이해\n\n- 🎉 여러분은 RAG의 비법노트에 도달했습니다.\n- 🔄 RAG의 내용을 반복하여 이해를 높이세요.\n- 📂 실습 파일을 통해 RAG 프로세스를 이해하는 것이 중요합니다.\n- 🧐 RAG가 무엇인지와 구현 방법을 알아야 합니다.\n- 🎨 RAG 사용 목적을 그림으로 설명할 예정입니다.\n- ❓ RAG의 필요성을 채찍 PT와 비교하여 설명할 것입니다.\n- 📄 최신 정보를 제공하는 것이 RAG의 주요 목적입니다.\n- 📊 GPD는 사전학습된 정보에 의존하므로 정보가 오래되면 정확한 답변을 하지 못합니다.\n- 🔗 RAG는 주어진 정보를 참고하여 답변하는 방식으로 프롬프트가 바뀝니다.\n- 📚 GPD는 사전학습된 정보에 기반하여만 답변할 수 있습니다.\n- ⏳ 오래된 사전학습 정보는 정확한 답변을 어렵게 만듭니다.\n- 🔍 프롬프트가 변경되어 주어진 정보를 바탕으로 질문에 답변하도록 합니다.\n- 📄 PDF 문서에서 관련 정보를 검색하여 답변할 수 있습니다.\n- ⚠️ 많은 정보를 입력할 경우 비용이 증가하고 정보 탐색이 어려워질 수 있습니다.\n- 🔑 관련성 있는 정보만 제공하는 것이 최선입니다.\n- 📝 문서에서 텍스트를 긁어오는 방식으로 정보를 로드합니다.\n- 🔍 질문에 대한 유사도 검색을 통해 관련 단락을 추출합니다.\n- 📈 유사도가 높은 단락을 검색해 최상위 결과를 제공합니다.\n- 🧮 인베딩은 문장을 수학적 표현으로 바꾸어 정보 검색을 향상시킵니다.\n- 💾 인베딩 후 변환된 데이터를 저장하는 과정이 필요합니다.\n- 🔍 저장된 데이터는 검색어를 통해 관련 문서를 찾는 데 사용됩니다.\n- 📽️ 이번 영상은 사전 단계까지 살펴보았고, 다음 영상에서 후반부를 다룰 예정입니다.\n\n### RECOMMENDED QUESTIONS:\n1. RAG 시스템이 기존 GPD와 어떻게 차별화되는지 설명할 수 있나요?\n2. RAG의 인베딩 과정에서 발생하는 비용은 어떤 요소에 의해 결정되나요?\n3. PDF 문서에서 정보를 검색할 때 유사도 계산은 어떻게 이루어지나요?")

In [ ]:
url = "https://youtu.be/AA621UofTUA?si=gn4XutRMWUDSYLFL"

In [ ]:
text = """Please summarize the sentence according to the following FINAL REQUEST. 
FINAL REQUEST:
1. The provided summary sections are partial summaries of one document. Please combine them into a single cohesive summary.
2. Summarize the main points in bullet points in KOREAN.
3. Each summarized sentence must start with a single emoji that fits the meaning of the sentence.
4. Use various emojis to make the summary more interesting, but keep it concise and relevant.
5. Focus on identifying and presenting only one main topic and one overall summary for the document.
6. Avoid redundant or repeated points, and ensure that the summary covers all key ideas without introducing multiple conclusions or topics.

CONTEXT: 
{context}

FINAL SUMMARY:"""

In [ ]:
print("Please summarize the sentence according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points in KOREAN.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. Translate the summary into KOREAN if it is written in ENGLISH.\n5. DO NOT translate any technical terms.\n6. DO NOT include any unnecessary information.\n\nCONTEXT:\n{context}\n\nSUMMARY:"\n")

In [ ]:
print("Please summarize the sentence according to the following FINAL REQUEST. \nFINAL REQUEST:\n1. The provided summary sections are partial summaries of one document. Please combine them into a single cohesive summary.\n2. Summarize the main points in bullet points in KOREAN, but DO NOT translate any technical terms.\n3. Each summarized sentence must start with a single emoji that fits the meaning of the sentence.\n4. Use various emojis to make the summary more interesting, but keep it concise and relevant.\n5. Focus on identifying and presenting only one main topic and one overall summary for the document.\n6. Avoid redundant or repeated points, and ensure that the summary covers all key ideas without introducing multiple conclusions or topics.\n7. Please refer to each summary and indicate the key topic.\n8. If the original text is in English, we have already provided a summary translated into Korean, so please do not provide a separate translation.\n\nCONTEXT: \n{context}\n\nFINAL SUMMARY:")

In [ ]:
text = "[FINAL SUMMARY]\n• 📚 레그의 비법노트는 반복 학습과 레그에 대한 깊은 이해가 필요합니다.\n• 🖼️ RAG의 도입은 최신 정보를 제공하기 위해 중요하며, 좌측의 레그와 우측의 기존 방법을 비교해 설명합니다.\n• 📈 프롬프트의 변화와 정보의 정확성이 중요하며, 사전학습된 정보는 시간이 지나면 신뢰성이 떨어집니다.\n• 🔑 유사도 검색을 통해 관련 단락을 찾아내고, 텍스트 스플리터와 인베딩 과정을 통해 정보의 정확성을 확보합니다.\n• 💰 많은 입력 정보는 비용 증가와 정보 탐색의 어려움을 초래할 수 있습니다.\n\n[RECOMMEND QUESTIONS]\n1. RAG의 도입이 왜 중요한가요?\n2. 유사도 검색에서 어떤 방법으로 관련 단락을 찾나요?\n3. 입력 정보가 많을 때 발생할 수 있는 문제는 무엇인가요?""

In [ ]:
print(text.split("[FINAL SUMMARY]")[1].split("[RECOMMEND QUESTIONS]")[0])

In [ ]:
text.split("[FINAL SUMMARY]")[1].split("[RECOMMEND QUESTIONS]")[1].split("\n")[1].split(".")[1].strip()

In [ ]:
scripts = [{"start": 0.0, "end": 29.42, "text": " 여러분 안녕하세요 드디어 레그의 비법노트에 레그 파트까지 오시느라 정말 고생 많으셨습니다 레그의 전반적인 내용을 먼저 한번 들어보시고요 그리고 잘 이해가 안되면 또 반복해서 들어보실 수 있으니까 반복해서 들어보시고 그리고 더 중요한 거는 이 실습 파일들을 여러분들이 반복해서 보시면서 계속 레그에 대한 프로세스 이해가 있어야 그 다음에 다시 역으로 돌아가서 우리가 이런 것들을 살펴볼 거에요 아프파서랑 모델 메모리 체인들 이런 것들을 쭉 살펴볼 때 역으로 더 이해가 잘 되실 거라는 생각이 들더라구요"}, {"start": 30.0, "end": 42.5, "text": " 우리가 여기 처음부터 다 하고 가려면 너무 시간이 오래 걸리니까 이번 시간에는 레그를 좀 깊게 다뤄보기 보다는 일단은 레그가 뭔지 그리고 어떤 식으로 구현하는지 대충 감을 잡는다 그런 생각으로 오시면 됩니다."}, {"start": 43.21, "end": 60.03, "text": " 저희가 먼저 여기 레그의 베이식, 이 정도 수준에서 먼저 볼 건데요. 먼저 그러려면은 우리가 레그에 대한 이해가 필요할 것 같아요. 그래서 제가 좀 그림으로 그려왔어요. 제가 그림으로 그리는 걸 되게 좋아하는데 이 레그라는 걸 도대체 왜 쓰느냐, 우리가 그 강의 초반에도 말씀드렸잖아요."}, {"start": 60.0, "end": 74.72, "text": " 레그를 쓰는 목적에 대해서 다시 한 번만 짚고 넘어가 볼게요. 우리가 레그를 안 쓰고 채찍 PT 같은 걸 통해서 질문을 합니다. 우리가 일반적으로 채찍 PT에서 쓰는 거는 이런 방식이거든요. 여기에 여러분들이 이러한 퀘션들을 넣어줘요."}, {"start": 76.47, "end": 89.55, "text": " 프롬프트로 들어가죠. 당신은 친절한 답변하는 AI 어시스턴트입니다. 이런 것들이 들어가고요. 그 다음에 좀 더 확대해서 보여드리면 이렇게 들어가죠. 당신은 친절한 답변을 하는 어시스턴트."}, {"start": 90.82, "end": 117.02, "text": " 사용자의 질문이 여기 들어와요. 그러면 우리가 스트리밋으로 구현한 것처럼 요거에 대해서 프롬프트 완성을 해서 결국에는 이 LLM한테 전달이 된다는 거예요. 우리가 그걸 GPT를 쓸 수도 있고 아니면 뭐 클로드라는 모델을 쓸 수도 있고 라마3라는 오픈모델을 쓸 수도 있고요. 어쨌든 이걸 넣어서 우리가 얻는 답변은 뭐냐면 삼성전자가 자체 개발한 AI의 이름은 요거는 제가 채찍PT한테 물어본 거거든요. 답변을 이제 이런 식으로 준다는 거예요."}]

In [ ]:
scripts = [script["text"] for script in scripts]

In [ ]:
scripts[1]

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0",torch_dtype=torch.bfloat16,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")


In [ ]:
# 더 명확한 한국어 프롬프트 템플릿
prompt_template = """이 문서의 오탈자와 어색한 표현을 전문 교정자의 입장에서 자연스럽게 수정해주세요.
다음과 같은 사항을 중점적으로 검토해주세요:
1. 문맥에 맞지 않는 단어를 수정
2. 영어 발음은 알파벳으로 변경
3. 기술적인 용어는 원어로 변경
4. 원본 텍스트의 구조를 수정하지 말 것

원문: {prompt}

교정 결과:"""

# 입력 텐서 생성 및 GPU 이동
text = scripts[0]
model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors="pt")
model_inputs = {k: v.to("cuda") for k, v in model_inputs.items()}

# 생성 파라미터 설정
generation_config = {
    "max_new_tokens": 256,
    "temperature": 0.7,        # 창의성 조절 (0.0-1.0)
    "top_p": 0.9,             # nucleus sampling
    "do_sample": True,        # 다양한 출력을 위해 샘플링 사용
    "num_return_sequences": 1, # 생성할 결과 수
    "top_k": 50,              # top-k sampling
    "repetition_penalty": 1.2, # 반복 방지
    "no_repeat_ngram_size": 3  # n-gram 반복 방지
}

# 텍스트 생성
outputs = model.generate(**model_inputs, **generation_config)

# 결과 디코딩
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# 원본과 교정본 비교 출력
print("=== 원본 ===")
print(text)
print("\n=== 교정본 ===")
print(output_text)

In [ ]:
from pytubefix import YouTube

yt = YouTube(url)
audio_stream = yt.streams.filter(only_audio=True).first()

In [ ]:
# from faster_whisper import WhisperModel, BatchedInferencePipeline
# from tqdm import tqdm


# model = WhisperModel(
#     "large-v3", device="cuda", compute_type="bfloat16"
# )
# model = BatchedInferencePipeline(model=model)  # 배치 모델일 경우
# print("Whisper 모델 초기화 완료")


# segments, info = model.transcribe(
#     audio_stream.url,
#     batch_size=64,  # 배치 모델인 경우
#     repetition_penalty=1.5,
#     beam_size=10,
#     patience=2,
#     no_repeat_ngram_size=4,
# )

In [ ]:
from faster_whisper import WhisperModel, BatchedInferencePipeline
from tqdm import tqdm
import soundfile as sf
import math
import requests
import tempfile
import os
import concurrent.futures
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import threading
import ffmpeg

class ProgressBar:
    def __init__(self, total_size, desc="Downloading"):
        pbar = tqdm(total=total_size, unit="iB", unit_scale=True, desc=desc)
        lock = threading.Lock()

    def update(self, size):
        with lock:
            pbar.update(size)

    def close(self):
        pbar.close()

def create_session():
    session = requests.Session()
    retry = Retry(
        total=5,
        backoff_factor=0.1,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=100,
        pool_maxsize=100
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def download_chunk(args):
    url, start, end, chunk_number, temp_dir, progress_bar = args
    
    headers = {"Range": f"bytes={start}-{end}"}
    session = create_session()
    
    try:
        response = session.get(url, headers=headers, stream=True)
        chunk_path = os.path.join(temp_dir, f"chunk_{chunk_number:04d}")
        
        with open(chunk_path, "wb") as f:
            for data in response.iter_content(chunk_size=8192):
                size = f.write(data)
                progress_bar.update(size)
        
        return chunk_path, chunk_number
    except Exception as e:
        print(f"Error downloading chunk {chunk_number}: {str(e)}")
        return None, chunk_number

def parallel_download(url, temp_dir, num_chunks=10):
    session = create_session()
    response = session.head(url)
    total_size = int(response.headers.get("content-length", 0))
    
    if total_size == 0:
        raise ValueError("Could not determine file size")
    
    chunk_size = total_size // num_chunks
    chunks = []
    
    for i in range(num_chunks):
        start = i * chunk_size
        end = start + chunk_size - 1 if i < num_chunks - 1 else total_size - 1
        chunks.append((start, end))
    
    progress_bar = ProgressBar(total_size, "Parallel downloading")
    
    download_args = [
        (url, start, end, i, temp_dir, progress_bar)
        for i, (start, end) in enumerate(chunks)
    ]
    
    chunk_paths = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_chunks) as executor:
        futures = executor.map(download_chunk, download_args)
        chunk_paths = [(path, num) for path, num in futures if path is not None]
    
    progress_bar.close()
    
    chunk_paths.sort(key=lambda x: x[1])
    output_path = os.path.join(temp_dir, "complete_audio.mp4")
    
    with open(output_path, "wb") as outfile:
        for chunk_path, _ in chunk_paths:
            with open(chunk_path, "rb") as infile:
                outfile.write(infile.read())
            os.remove(chunk_path)
    
    return output_path

def convert_to_wav(input_path, output_path):
    """MP4를 WAV로 변환"""
    try:
        stream = ffmpeg.input(input_path)
        stream = ffmpeg.output(stream, output_path, 
                             acodec="pcm_s16le", 
                             ar="16000",
                             ac="1")
        ffmpeg.run(stream, capture_stdout=True, capture_stderr=True)
        return True
    except ffmpeg.Error as e:
        print("FFmpeg error:", e.stderr.decode())
        return False

def process_audio_chunk(chunk_data):
    """개별 오디오 청크 처리"""
    model, audio_path, start_time, duration = chunk_data
    try:
        segments, info = model.transcribe(
            audio_path,
            beam_size=5,
            batch_size=32,
            word_timestamps=True,
            initial_prompt=None
        )
        
        # segments를 리스트로 변환하고 시간 조정
        chunk_segments = []
        for segment in segments:
            segment_dict = {
                "start": segment.start + start_time,
                "end": segment.end + start_time,
                "text": segment.text,
                "words": [
                    {
                        "start": word.start + start_time,
                        "end": word.end + start_time,
                        "word": word.word,
                        "probability": word.probability
                    }
                    for word in segment.words
                ]
            }
            chunk_segments.append(segment_dict)
        
        return chunk_segments
    except Exception as e:
        print(f"Error processing chunk at {start_time}: {str(e)}")
        return []

def process_with_progress(url, model, chunk_duration=30, num_download_chunks=10):
    """
    전체 처리 프로세스 관리
    """
    with tempfile.TemporaryDirectory() as temp_dir:
        print("Starting parallel download...")
        mp4_path = parallel_download(url, temp_dir, num_download_chunks)
        print("Download complete!")
        
        # MP4를 WAV로 변환
        wav_path = os.path.join(temp_dir, "audio.wav")
        if not convert_to_wav(mp4_path, wav_path):
            raise Exception("Failed to convert audio to WAV format")
        
        # WAV 파일 정보 읽기
        wav_info = sf.info(wav_path)
        total_duration = wav_info.duration
        
        # 청크 계산
        total_chunks = math.ceil(total_duration / chunk_duration)
        
        # 진행률 표시
        pbar = tqdm(total=total_chunks, desc="Processing audio chunks")
        
        # 청크 처리를 위한 데이터 준비
        chunks_data = []
        for i in range(total_chunks):
            start_time = i * chunk_duration
            chunk_wav_path = os.path.join(temp_dir, f"chunk_{i}.wav")
            
            # 청크 추출
            duration = min(chunk_duration, total_duration - start_time)
            stream = ffmpeg.input(wav_path, ss=start_time, t=duration)
            stream = ffmpeg.output(stream, chunk_wav_path, 
                                 acodec="pcm_s16le", 
                                 ar="16000",
                                 ac="1")
            ffmpeg.run(stream, quiet=True)
            
            chunks_data.append((model, chunk_wav_path, start_time, duration))
        
        # 청크 처리 및 결과 수집
        all_segments = []
        for chunk_data in chunks_data:
            segments = process_audio_chunk(chunk_data)
            all_segments.extend(segments)
            pbar.update(1)
            
            # 사용한 청크 파일 삭제
            if os.path.exists(chunk_data[1]):
                os.remove(chunk_data[1])
        
        pbar.close()
        
    return all_segments

# 모델 초기화
model = WhisperModel(
    "large-v3", 
    device="cuda", 
    compute_type="float16"  # bfloat16 대신 float16 사용
)
print("Whisper 모델 초기화 완료")
model = BatchedInferencePipeline(model=model)

# 트랜스크립션 실행
segments = process_with_progress(
    audio_stream.url,
    model,
    chunk_duration=30,
    num_download_chunks=10
)

# 결과 저장
for i, segment in enumerate(segments):
    print(f"{segment["start"]:.2f} -> {segment["end"]:.2f}: {segment["text"]}")

In [ ]:
# url = "https://youtu.be/AA621UofTUA?si=gn4XutRMWUDSYLFL"

# from faster_whisper import WhisperModel
# from tqdm import tqdm
# import numpy as np
# import soundfile as sf
# import tempfile
# import os
# import ffmpeg
# import subprocess
# from yt_dlp import YoutubeDL
# import io

# def get_audio_stream(url):
#     """URL에서 오디오 스트림 정보를 가져옵니다."""
#     ydl_opts = {
#         "format": "bestaudio/best",
#         "quiet": True,
#         "no_warnings": True,
#         "extract_audio": True
#     }
    
#     with YoutubeDL(ydl_opts) as ydl:
#         info = ydl.extract_info(url, download=False)
#         audio_url = info["url"]
#         duration = info.get("duration", 0)
        
#         return audio_url, duration

# def process_stream_with_progress(url, model, chunk_duration=30):
#     """
#     스트리밍 방식으로 오디오를 처리합니다.
    
#     Parameters:
#     - url: 오디오 URL
#     - model: WhisperModel 인스턴스
#     - chunk_duration: 각 청크의 길이(초)
#     """
#     # 스트림 URL 가져오기
#     audio_url, total_duration = get_audio_stream(url)
    
#     # ffmpeg 명령어 설정
#     ffmpeg_cmd = [
#         "ffmpeg",
#         "-i", audio_url,
#         "-f", "wav",
#         "-ar", "16000",
#         "-ac", "1",
#         "-hide_banner",
#         "-loglevel", "error",
#         "pipe:1"
#     ]
    
#     # 진행률 표시 설정
#     total_chunks = int(np.ceil(total_duration / chunk_duration))
#     pbar = tqdm(total=total_chunks, desc="Processing audio chunks")
    
#     # 결과 저장용 리스트
#     all_segments = []
    
#     try:
#         # ffmpeg 프로세스 시작
#         process = subprocess.Popen(
#             ffmpeg_cmd,
#             stdout=subprocess.PIPE,
#             bufsize=10**8  # 버퍼 크기 설정
#         )
        
#         # 임시 디렉토리 생성
#         with tempfile.TemporaryDirectory() as temp_dir:
#             chunk_size = int(16000 * chunk_duration * 2)  # 16000Hz * seconds * 2 bytes per sample
#             chunk_number = 0
            
#             while True:
#                 # 청크 읽기
#                 audio_chunk = process.stdout.read(chunk_size)
#                 if not audio_chunk:
#                     break
                
#                 # 청크를 임시 파일로 저장
#                 chunk_path = os.path.join(temp_dir, f"chunk_{chunk_number}.wav")
#                 with open(chunk_path, "wb") as f:
#                     # WAV 헤더 작성
#                     f.write(b"RIFF")
#                     f.write((chunk_size + 36).to_bytes(4, "little"))
#                     f.write(b"WAVE")
#                     f.write(b"fmt ")
#                     f.write((16).to_bytes(4, "little"))
#                     f.write((1).to_bytes(2, "little"))  # PCM
#                     f.write((1).to_bytes(2, "little"))  # Mono
#                     f.write((16000).to_bytes(4, "little"))  # Sample rate
#                     f.write((32000).to_bytes(4, "little"))  # Byte rate
#                     f.write((2).to_bytes(2, "little"))  # Block align
#                     f.write((16).to_bytes(2, "little"))  # Bits per sample
#                     f.write(b"data")
#                     f.write(len(audio_chunk).to_bytes(4, "little"))
#                     f.write(audio_chunk)
                
#                 try:
#                     # 청크 처리
#                     segments, _ = model.transcribe(
#                         chunk_path,
#                         beam_size=5,
#                         batch_size=32,
#                         word_timestamps=True,
#                         condition_on_previous_text=True
#                     )
                    
#                     # 시간 오프셋 조정 및 세그먼트 저장
#                     time_offset = chunk_number * chunk_duration
#                     for segment in segments:
#                         segment_dict = {
#                             "start": segment.start + time_offset,
#                             "end": segment.end + time_offset,
#                             "text": segment.text,
#                             "words": [
#                                 {
#                                     "start": word.start + time_offset,
#                                     "end": word.end + time_offset,
#                                     "word": word.word,
#                                     "probability": word.probability
#                                 }
#                                 for word in segment.words
#                             ]
#                         }
#                         all_segments.append(segment_dict)
                
#                 except Exception as e:
#                     print(f"Error processing chunk {chunk_number}: {str(e)}")
                
#                 finally:
#                     # 임시 파일 삭제
#                     if os.path.exists(chunk_path):
#                         os.remove(chunk_path)
                
#                 # 진행률 업데이트
#                 pbar.update(1)
#                 chunk_number += 1
    
#     finally:
#         pbar.close()
#         if process.poll() is None:
#             process.terminate()
#             process.wait()
    
#     return all_segments

# # 모델 초기화
# model = WhisperModel(
#     "large-v3", 
#     device="cuda", 
#     compute_type="float16"
# )
# print("Whisper 모델 초기화 완료")

# # 트랜스크립션 실행
# segments = process_stream_with_progress(
#     url,  # 유튜브 URL
#     model,
#     chunk_duration=30
# )

# # 결과 출력
# for segment in segments:
#     print(f"{segment["start"]:.2f} -> {segment["end"]:.2f}: {segment["text"]}")

In [ ]:
from langchain.docstore.document import Document

In [ ]:
import json
with open("script.json","r",encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
documents = [
            Document(page_content="\n".join([t["text"] for t in data]))
        ]

In [ ]:
documents[0].page_content

In [ ]:
import tiktoken

def calculate_tokens(text, model="gpt-4o-mini"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

In [ ]:
calculate_tokens(documents[0].page_content)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
summarize_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000, chunk_overlap=500
        )

In [ ]:
split_docs = summarize_splitter.split_documents(documents)

In [ ]:
len(split_docs)

In [ ]:
type(split_docs[0])

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chat_models import ChatOpenAI
from langchain import hub
summary_prompt = hub.pull("teddynote/summary-stuff-documents-korean")
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7, streaming=True)
summary_chain = create_stuff_documents_chain(llm, summary_prompt)

In [ ]:
sumaries = []
for split_doc in split_docs:
    print(type(split_doc.page_content))
    partial_summary = summary_chain.invoke({"context": [split_doc]})
    sumaries.append(partial_summary)

In [ ]:
partial_summary = Document(page_content= "\n".join(sumaries))

In [ ]:
SUMMARY_RESULT = summary_chain.invoke(
                {"context": partial_summaries_doc}
            )

In [ ]:
len(SUMMARY_RESULT.split("\n"))

In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_qa_with_sources_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
from pytubefix import YouTube
import asyncio
import torch
from faster_whisper import WhisperModel

In [ ]:
load_dotenv()  # .env 파일에서 환경 변수 로드

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
def get_video_info(url):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    return {
        "title": yt.title,
        "audio_url": audio_stream.url if audio_stream else None
    }


In [ ]:
video_url = "https://www.youtube.com/shorts/a--NSC19MXM"
video_info = get_video_info(video_url)
print(f"Video Title: {video_info["title"]}")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

In [ ]:
whisper_model = WhisperModel("large-v3", device=device, compute_type=compute_type)

def transcribe_audio(audio_url):
    segments, info = whisper_model.transcribe(audio_url)
    transcript = [{"text": segment.text, "start": segment.start, "end": segment.end} for segment in segments]
    return {"script": transcript, "language": info.language}

In [ ]:
transcript = transcribe_audio(video_info["audio_url"])
print(f"Transcript Language: {transcript["language"]}")
print(f"First few lines of transcript: {transcript["script"][:3]}")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
summary_prompt = hub.pull("teddynote/summary-stuff-documents-korean")
llm = ChatOpenAI(
            model_name="gpt-4o-mini",
            temperature=0.7,
            streaming=True,
        )

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
from langchain_community.document_loaders import JSONLoader, TextLoader
docs = TextLoader("script.txt").load()

In [ ]:
docs

In [ ]:
from langchain_core.documents import Document
document = [Document(page_content="\n".join([t["text"] for t in transcript["script"]]))]

In [ ]:
summary_chain = create_stuff_documents_chain(llm,summary_prompt)
result = await summary_chain.ainvoke({"context": document})

In [ ]:
result

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
result.strip().split("\n")

In [ ]:
documents = text_splitter.create_documents([t["text"] for t in transcript["script"]])
for doc in documents:
    doc.page_content += "\n" + summary.strip()

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
you_url = "https://youtube.com/shorts/a--NSC19MXM?si=yiun-HK_7wX1sNvL"

In [ ]:
import requests
import os
from dotenv import load_dotenv


# RunPod RUNSYNC 엔드포인트 URL
url = "https://api.runpod.ai/v2/uq96boxkzy99ev/runsync"

# FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터 (내부적으로 사용할 파라미터 설정)
body = {"input":{
    "api":{
        "method":"POST",
        "endpoint":"/ping",
    },
    "payload":{},
}}
# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# RUNSYNC 요청 보내기
response = requests.post(url, json=body, headers=headers)

# 응답 확인
if response.status_code == 200:
    print("Response:", response.json())
else:
    print(f"Error {response.status_code}: {response.text}")


In [ ]:
import requests

# 작업 ID (작업 완료된 job ID)
job_id = response.json()["id"]

# RunPod API STATUS 엔드포인트 URL
status_url = f"https://api.runpod.ai/v2/wm1xrz07all039/status/{job_id}"


# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}"
}

# 작업 상태 및 결과 확인 요청 보내기
response = requests.get(status_url, headers=headers)

# 응답 확인
if response.status_code == 200:
    job_result = response.json()
    if job_result.get("status") == "COMPLETED":
        print("Job Completed! Result:", job_result.get("output"))
    else:
        print(f"Job Status: {job_result.get("status")}")
else:
    print(f"Error {response.status_code}: {response.text}")


In [ ]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
endpoint_id = os.getenv("RUNPOD_ENDPOINT_ID")

# RunPod RUNSYNC 엔드포인트 URL
url = f"https://api.runpod.ai/v2/{endpoint_id}/runsync"
you_url = "https://youtu.be/omEk2BNDt1I?si=xjtbYANtlux5CTfB"


# FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터
payload = {
    "input": {
        "endpoint": "/get_title_hash",
        "method": "GET",
        # "headers": {"x-session-id": "1234asdf"},
        "params": {"url": you_url},
    }
}

# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# RUNSYNC 요청 보내기
response = requests.post(url, json=payload, headers=headers)

# 응답 확인
if response.status_code == 200:
    print("Response:", response.json())
else:
    print(f"Error {response.status_code}: {response.text}")


In [ ]:
# import requests
# import os
# from dotenv import load_dotenv

# # Load environment variables
# load_dotenv()
# api_key = os.getenv("RUNPOD_API_KEY")
# endpoint_id = os.getenv("RUNPOD_ENDPOINT_ID")

# # RunPod RUNSYNC 엔드포인트 URL
# url = f"https://api.runpod.ai/v2/{endpoint_id}/runsync"

# # FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터
# payload = {
#     "input": {
#         "endpoint": "/get_script_summary",
#         "method": "GET",
#         "headers": {"x-session-id": "1234asdf"},
#         "params": {"url": you_url},
#     }
# }

# # 요청 헤더에 API 키 추가
# headers = {
#     "Authorization": f"Bearer {api_key}",
#     "Content-Type": "application/json"
# }

# # RUNSYNC 요청 보내기
# response = requests.post(url, json=payload, headers=headers)

# # 응답 확인
# if response.status_code == 200:
#     print("Response:", response.json())
# else:
#     print(f"Error {response.status_code}: {response.text}")


In [ ]:
import requests
import os
import time
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
endpoint_id = os.getenv("RUNPOD_ENDPOINT_ID")

# RunPod RUNSYNC 엔드포인트 URL
url = f"https://api.runpod.ai/v2/{endpoint_id}/run"

# FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터
payload = {
    "input": {
        "endpoint": "/get_script_summary",
        "method": "GET",
        "headers": {"x-session-id": "1234asdf"},
        "params": {"url": you_url},
    }
}

# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# RUNSYNC 요청 보내기
response = requests.post(url, json=payload, headers=headers)

# 응답 확인
if response.status_code == 200:
    result = response.json()
    print("Initial Response:", result)
    
    if result.get("status") in ["IN_PROGRESS","IN_QUEUE"]:
        job_id = result.get("id")
        status_url = f"https://api.runpod.ai/v2/{endpoint_id}/status/{job_id}"
        
        while True:
            status_response = requests.get(status_url, headers=headers)
            if status_response.status_code == 200:
                status_data = status_response.json()
                print(f"Current status: {status_data.get("status")}")
                
                if status_data.get("status") == "COMPLETED":
                    print(f"결과값:{status_data}")
                    result_url = f"https://api.runpod.ai/v2/{endpoint_id}/result/{job_id}"
                    result_response = requests.get(result_url, headers=headers)
                    
                    if result_response.status_code == 200:
                        final_result = result_response.json()
                        print("Final Result:", final_result)
                        break
                    else:
                        print(f"Error fetching results: {result_response.status_code}")
                        print(f"Error message: {result_response.text}")
                        break
                elif status_data.get("status") == "FAILED":
                    print("Job failed")
                    break
            else:
                print(f"Error checking status: {status_response.status_code}")
                print(f"Error message: {status_response.text}")
                break
            
            time.sleep(5)  # 5초 대기 후 다시 상태 확인
    else:
        print("Job completed immediately")
        print("Final Result:", result)
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
import requests
import os
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
endpoint_id = os.getenv("RUNPOD_ENDPOINT_ID")

# RunPod RUNSYNC 엔드포인트 URL
url = f"https://api.runpod.ai/v2/{endpoint_id}/status/{job_id}"
url2 = f"https://api.runpod.ai/v2/{endpoint_id}/result/{job_id}"

# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

while True:
    # RUNSYNC 요청 보내기
    response = requests.get(url,headers=headers)

    # 응답 확인
    if response.status_code == 200:
        if response.json().get("status") == "COMPLETED":
            response = requests.get(url2,headers=headers)
            break
        else:
            print(f"Job status: {response.json().get("status")}")
    else:
        print(f"Error {response.status_code}: {response.text}")
    time.sleep(5)


In [ ]:
response.text

In [ ]:
import json
RUNPOD_API_URL = f"https://api.runpod.ai/v2/{endpoint_id}/runsync"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}
payload = {
    "input": {
        "endpoint": "/rag_stream_chat",
        "method": "POST",
        "headers": {"x-session-id": "1234asdf"},
        "params": {"prompt": "영상의 주제가 뭔가요?"},
    }
}

response = requests.post(
    RUNPOD_API_URL, headers=headers, json=payload, stream=True
)

# for chunk in response.iter_content(chunk_size=None):
#     if chunk:
#         chunk_data = chunk.decode("utf-8").strip()
#         if chunk_data.startswith("data: "):
#             chunk_content = chunk_data[6:]
#             if chunk_content == "[DONE]":
#                 break
#             try:
#                 content = json.loads(chunk_content)
#                 print("Stream content:", content)
#             except json.JSONDecodeError:
#                 print("Invalid JSON:", chunk_content)


In [ ]:
answer = ""
for chunk in response.json().get("output"):
    answer += chunk.get("content")
    print(answer)

In [ ]:
with open("script.txt","r") as f:
    lines = f.readlines()

In [ ]:
context = "".join(lines)

In [ ]:
print(context)

In [ ]:
from pytubefix import YouTube
url = "https://www.youtube.com/watch?v=yF_YIxxjWU4"
yt = YouTube(url, use_po_token=True)

In [ ]:
yt.title